# Backtesting SARIMAX and ARIMA models

## Libraries

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/home/javi/Documents/GitHub/skforecast') # Ubuntu
sys.path.insert(1, '/home/ximo/Documents/GitHub/skforecast')
%config Completer.use_jedi = False

SARIMAX (Seasonal Auto-Regressive Integrated Moving Average with eXogenous factors) is a generalization of the ARIMA model that allows incorporating seasonality and exogenous variables. This model have a total of 6 hyperparameters that must specified when training the model:

+ p: Trend autoregression order.

+ d: Trend difference order.

+ q: Trend moving average order.

+ P: Seasonal autoregressive order.

+ D: Seasonal difference order.

+ Q: Seasonal moving average order.

+ m: The number of time steps for a single seasonal period.

One way to find the best values is by using grid search. The `grid_search_sarimax` function of the *skforecast.model_selection_statsmodels* module is a wrapper that automates this process using the [SARIMAX](https://www.statsmodels.org/stable/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html#statsmodels.tsa.statespace.sarimax.SARIMAX)   implementation available in the statsmodels library.

In [2]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skforecast.model_selection_statsmodels import backtesting_sarimax

## Data

In [3]:
# Download data
# ==============================================================================
url = ('https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/data/h2o.csv')
data = pd.read_csv(url, sep=',', header=0, names=['y', 'datetime'])

# Data preprocessing
# ==============================================================================
data['datetime'] = pd.to_datetime(data['datetime'], format='%Y/%m/%d')
data = data.set_index('datetime')
data = data.asfreq('MS')
data = data['y']
data = data.sort_index()

# Split data in train snd backtest
# ==============================================================================
n_backtest = 36*3  # Last 9 years are used for backtest
data_train = data[:-n_backtest]
data_backtest = data[-n_backtest:]

## Backtest

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
metric, predictions_backtest = backtesting_sarimax(
                                    y = data,
                                    order = (12, 1, 1),
                                    seasonal_order = (0, 0, 0, 0),
                                    initial_train_size = len(data_train),
                                    fixed_train_size = False,
                                    steps = 7,
                                    metric = 'mean_absolute_error',
                                    refit = False,
                                    verbose = True,
                                    fit_kwargs = {'maxiter': 250, 'disp': 0},
                                )


Number of observations used for training: 96
Number of observations used for backtesting: 108
    Number of folds: 16
    Number of steps per fold: 7
    Last fold only includes 3 observations.


In [6]:
print(f"Error backtest: {metric}")
print(predictions_backtest.head(4).to_markdown())

Error backtest: 0.055440709178810936
|                     |   predicted_mean |   lower y |   upper y |
|:--------------------|-----------------:|----------:|----------:|
| 1999-07-01 00:00:00 |         0.734693 |  0.650096 |  0.81929  |
| 1999-08-01 00:00:00 |         0.751835 |  0.660703 |  0.842967 |
| 1999-09-01 00:00:00 |         0.865333 |  0.768144 |  0.962522 |
| 1999-10-01 00:00:00 |         0.832345 |  0.730306 |  0.934385 |
